In [1]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import requests
import xml.etree.ElementTree as ET
from scraper import *
import numpy as np
import json
import random
import time
import datetime as dt
%load_ext autoreload
%autoreload 2

# display all pandas columns

pd.set_option('display.max_columns', 100)

# Prepare for the right formats

The formats for the graph in D3 are specific. We need to make sure we follow the same.

In [2]:
df_people = pd.read_csv('data/people.csv', encoding='utf-8')

In [3]:
df_people.head()

,Active,AdditionalActivity,AdditionalMandate,CantonAbbreviation,CantonName,Citizenship,CouncilAbbreviation,CouncilName,DateElection,DateJoining,DateLeaving,DateOath,DateResignation,ID,IdPredecessor,Mandates,ParlGroupAbbreviation,ParlGroupFunctionText,ParlGroupName,PartyAbbreviation,PartyName,DateOfBirth,DateOfDeath,FirstName,GenderAsString,LastName,MaritalStatusText,MilitaryRankText,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,TitleText
0,False,Réducteur TSV de 1971 à 1983,"Prés. Féd. Romande des Sociolistes ch., Prés. ...",VD,Vaud,"Sullens (VD),Lutry (VD)",CN,Conseil national,1995-12-04T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1,NaN,"Conseiller communal de 1965 à 1982, Conseiller...",NaN,NaN,NaN,PSS,Parti socialiste suisse,1938-03-02T00:00:00,NaN,Pierre,m,Aguet,marié(e),Fourrier,F,NaN,Aguet,2200.0,1,Vaud,Pompaples,NaN
1,False,NaN,NaN,ZH,Zurich,"Kreuzlingen (TG),Fällanden (ZH)",CN,Conseil national,1979-11-26T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,2,NaN,NaN,NaN,NaN,NaN,PLR,PLR.Les Libéraux-Radicaux,1928-02-22T00:00:00,NaN,Heinz,m,Allenspach,NaN,NaN,D,NaN,Allenspach,2002.0,2,NaN,NaN,NaN
2,False,Zentralpräsident Schweiz. Skliverband 1985 bis...,NaN,LU,Lucerne,Hasle (LU),CN,Conseil national,1995-12-04T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,6,NaN,Grossrat 1967 bis 1983 (Präsident 1977),NaN,NaN,NaN,PLR,PLR.Les Libéraux-Radicaux,1931-01-27T00:00:00,NaN,Manfred,m,Aregger,marié(e),Adjudant sous-officier,D,5.0,Aregger,2004.0,6,Lucerne,Hasle,dipl. Bauing. HTL
3,False,NaN,NaN,BE,Berne,Tavannes (BE),CN,Conseil national,1979-11-26T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1928-03-04T00:00:00,NaN,Geneviève,f,Aubry,NaN,NaN,F,NaN,Aubry Geneviève,2005.0,7,NaN,NaN,NaN
4,False,NaN,NaN,BE,Berne,"Siselen (BE),Richterswil (ZH)",CN,Conseil national,1987-11-30T00:00:00,1987-11-30T00:00:00,1995-12-03T00:00:00,1987-11-30T00:00:00,1995-12-03T00:00:00,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1947-12-01T00:00:00,NaN,Rosmarie,f,Bär,NaN,NaN,D,NaN,Bär,2008.0,8,NaN,NaN,NaN


In [4]:
df_active_people = df_people[df_people['Active'] == True]
df_active_people = df_active_people.drop(['Active'], axis=1)
df_active_people.head()

,AdditionalActivity,AdditionalMandate,CantonAbbreviation,CantonName,Citizenship,CouncilAbbreviation,CouncilName,DateElection,DateJoining,DateLeaving,DateOath,DateResignation,ID,IdPredecessor,Mandates,ParlGroupAbbreviation,ParlGroupFunctionText,ParlGroupName,PartyAbbreviation,PartyName,DateOfBirth,DateOfDeath,FirstName,GenderAsString,LastName,MaritalStatusText,MilitaryRankText,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,TitleText
68,Vize-Präsident der Parlamentarischen Gruppe fü...,NaN,BS,Bâle-Ville,Bâle (BS),CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,74,NaN,"Weiterer Bürgerrat: von 1989 bis 1991, Grosser...",RL,Membre,Groupe libéral-radical,PLD,Parti libéral démocrate,1951-01-15T00:00:00,NaN,Christoph,m,Eymann,marié(e),Capitaine,D,3.0,Eymann,2287.0,74,Bâle-Ville,Bâle,Dr. iur.
85,Co-Präsident der Parlamentarischen Gruppe Elek...,NaN,AG,Argovie,"Rothrist (AG),Wattwil (SG)",CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,91,NaN,Legislative des Kantons (Grossrat): von März 1...,V,Membre,Groupe de l'Union démocratique du Centre,UDC,Union Démocratique du Centre,1953-10-31T00:00:00,NaN,Ulrich,m,Giezendanner,veuf/veuve,Soldat,D,3.0,Giezendanner,2292.0,91,Argovie,Rothrist,NaN
140,NaN,Präsident der SVP Schweiz: vom Januar 1996 bis...,ZH,Zurich,"Hinwil (ZH),Adelboden (BE)",CF,Conseil fédéral,2008-12-10T00:00:00,2008-12-10T00:00:00,NaN,2008-12-10T00:00:00,NaN,146,cef9ef51-4e74-4ef8-b9da-d39c33caef23,Exekutive der Gemeinde (Gemeinderat Hinwil): v...,NaN,NaN,NaN,UDC,Union Démocratique du Centre,1950-12-01T00:00:00,NaN,Ueli,m,Maurer,marié(e),Major,D,6.0,Maurer Ueli,2310.0,146,Zurich,Wetzikon (ZH),-
166,NaN,NaN,SG,St-Gall,Appenzell (AI),CE,Conseil des Etats,2015-11-15T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,172,NaN,Legislative der Gemeinde St. Gallen: von 1977 ...,S,Membre,Groupe socialiste,PSS,Parti socialiste suisse,1952-08-26T00:00:00,NaN,Paul,m,Rechsteiner,marié(e),NaN,D,NaN,Rechsteiner Paul,2143.0,172,St-Gall,St. Gall,lic. iur.
167,NaN,Vizepräsident SVP Bezirk Laufenburg: von 1985 ...,AG,Argovie,Oberhof (AG),CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,173,NaN,NaN,V,Membre,Groupe de l'Union démocratique du Centre,UDC,Union Démocratique du Centre,1942-05-07T00:00:00,NaN,Maximilian,m,Reimann,NaN,Capitaine,D,NaN,Reimann Maximilian,2147.0,173,Argovie,Laufenburg,Dr. iur.


In [5]:
new_index = np.arange(1, len(df_active_people['AdditionalMandate'])+1)
df_active_people.index = new_index
df_active_people.head()

,AdditionalActivity,AdditionalMandate,CantonAbbreviation,CantonName,Citizenship,CouncilAbbreviation,CouncilName,DateElection,DateJoining,DateLeaving,DateOath,DateResignation,ID,IdPredecessor,Mandates,ParlGroupAbbreviation,ParlGroupFunctionText,ParlGroupName,PartyAbbreviation,PartyName,DateOfBirth,DateOfDeath,FirstName,GenderAsString,LastName,MaritalStatusText,MilitaryRankText,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,TitleText
1,Vize-Präsident der Parlamentarischen Gruppe fü...,NaN,BS,Bâle-Ville,Bâle (BS),CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,74,NaN,"Weiterer Bürgerrat: von 1989 bis 1991, Grosser...",RL,Membre,Groupe libéral-radical,PLD,Parti libéral démocrate,1951-01-15T00:00:00,NaN,Christoph,m,Eymann,marié(e),Capitaine,D,3.0,Eymann,2287.0,74,Bâle-Ville,Bâle,Dr. iur.
2,Co-Präsident der Parlamentarischen Gruppe Elek...,NaN,AG,Argovie,"Rothrist (AG),Wattwil (SG)",CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,91,NaN,Legislative des Kantons (Grossrat): von März 1...,V,Membre,Groupe de l'Union démocratique du Centre,UDC,Union Démocratique du Centre,1953-10-31T00:00:00,NaN,Ulrich,m,Giezendanner,veuf/veuve,Soldat,D,3.0,Giezendanner,2292.0,91,Argovie,Rothrist,NaN
3,NaN,Präsident der SVP Schweiz: vom Januar 1996 bis...,ZH,Zurich,"Hinwil (ZH),Adelboden (BE)",CF,Conseil fédéral,2008-12-10T00:00:00,2008-12-10T00:00:00,NaN,2008-12-10T00:00:00,NaN,146,cef9ef51-4e74-4ef8-b9da-d39c33caef23,Exekutive der Gemeinde (Gemeinderat Hinwil): v...,NaN,NaN,NaN,UDC,Union Démocratique du Centre,1950-12-01T00:00:00,NaN,Ueli,m,Maurer,marié(e),Major,D,6.0,Maurer Ueli,2310.0,146,Zurich,Wetzikon (ZH),-
4,NaN,NaN,SG,St-Gall,Appenzell (AI),CE,Conseil des Etats,2015-11-15T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,172,NaN,Legislative der Gemeinde St. Gallen: von 1977 ...,S,Membre,Groupe socialiste,PSS,Parti socialiste suisse,1952-08-26T00:00:00,NaN,Paul,m,Rechsteiner,marié(e),NaN,D,NaN,Rechsteiner Paul,2143.0,172,St-Gall,St. Gall,lic. iur.
5,NaN,Vizepräsident SVP Bezirk Laufenburg: von 1985 ...,AG,Argovie,Oberhof (AG),CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,173,NaN,NaN,V,Membre,Groupe de l'Union démocratique du Centre,UDC,Union Démocratique du Centre,1942-05-07T00:00:00,NaN,Maximilian,m,Reimann,NaN,Capitaine,D,NaN,Reimann Maximilian,2147.0,173,Argovie,Laufenburg,Dr. iur.


In [6]:
parties = df_active_people['PartyAbbreviation'].unique()

party_color = []
for i in range(1,len(df_active_people['PartyAbbreviation'])+1):
    party_color.append(1 + np.where(parties == df_active_people['PartyAbbreviation'][i])[0][0])

df_active_people['PartyColor'] = pd.Series(party_color, index=df_active_people.index)
df_active_people.head()

,AdditionalActivity,AdditionalMandate,CantonAbbreviation,CantonName,Citizenship,CouncilAbbreviation,CouncilName,DateElection,DateJoining,DateLeaving,DateOath,DateResignation,ID,IdPredecessor,Mandates,ParlGroupAbbreviation,ParlGroupFunctionText,ParlGroupName,PartyAbbreviation,PartyName,DateOfBirth,DateOfDeath,FirstName,GenderAsString,LastName,MaritalStatusText,MilitaryRankText,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,TitleText,PartyColor
1,Vize-Präsident der Parlamentarischen Gruppe fü...,NaN,BS,Bâle-Ville,Bâle (BS),CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,74,NaN,"Weiterer Bürgerrat: von 1989 bis 1991, Grosser...",RL,Membre,Groupe libéral-radical,PLD,Parti libéral démocrate,1951-01-15T00:00:00,NaN,Christoph,m,Eymann,marié(e),Capitaine,D,3.0,Eymann,2287.0,74,Bâle-Ville,Bâle,Dr. iur.,1
2,Co-Präsident der Parlamentarischen Gruppe Elek...,NaN,AG,Argovie,"Rothrist (AG),Wattwil (SG)",CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,91,NaN,Legislative des Kantons (Grossrat): von März 1...,V,Membre,Groupe de l'Union démocratique du Centre,UDC,Union Démocratique du Centre,1953-10-31T00:00:00,NaN,Ulrich,m,Giezendanner,veuf/veuve,Soldat,D,3.0,Giezendanner,2292.0,91,Argovie,Rothrist,NaN,2
3,NaN,Präsident der SVP Schweiz: vom Januar 1996 bis...,ZH,Zurich,"Hinwil (ZH),Adelboden (BE)",CF,Conseil fédéral,2008-12-10T00:00:00,2008-12-10T00:00:00,NaN,2008-12-10T00:00:00,NaN,146,cef9ef51-4e74-4ef8-b9da-d39c33caef23,Exekutive der Gemeinde (Gemeinderat Hinwil): v...,NaN,NaN,NaN,UDC,Union Démocratique du Centre,1950-12-01T00:00:00,NaN,Ueli,m,Maurer,marié(e),Major,D,6.0,Maurer Ueli,2310.0,146,Zurich,Wetzikon (ZH),-,2
4,NaN,NaN,SG,St-Gall,Appenzell (AI),CE,Conseil des Etats,2015-11-15T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,172,NaN,Legislative der Gemeinde St. Gallen: von 1977 ...,S,Membre,Groupe socialiste,PSS,Parti socialiste suisse,1952-08-26T00:00:00,NaN,Paul,m,Rechsteiner,marié(e),NaN,D,NaN,Rechsteiner Paul,2143.0,172,St-Gall,St. Gall,lic. iur.,3
5,NaN,Vizepräsident SVP Bezirk Laufenburg: von 1985 ...,AG,Argovie,Oberhof (AG),CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,173,NaN,NaN,V,Membre,Groupe de l'Union démocratique du Centre,UDC,Union Démocratique du Centre,1942-05-07T00:00:00,NaN,Maximilian,m,Reimann,NaN,Capitaine,D,NaN,Reimann Maximilian,2147.0,173,Argovie,Laufenburg,Dr. iur.,2


In [7]:
df_active_people = df_active_people.fillna('NaN')
df_active_people = df_active_people.drop(['ID'], axis=1)

In [8]:
df_active_people['PartyAbbreviation'].unique()

array(['PLD', 'UDC', 'PSS', 'PDC', 'PLR', 'PES', 'pvl', 'PBD', 'PEV',
       'Lega', 'csp-ow', '-', 'MCG', 'BastA', 'PdT'], dtype=object)

In [9]:
df_active_people['PartyName'].unique()

array(['Parti libéral démocrate', 'Union Démocratique du Centre',
       'Parti socialiste suisse', 'Parti démocrate-chrétien suisse',
       'PLR.Les Libéraux-Radicaux', 'Parti écologiste suisse',
       "Parti vert'libéral", 'Parti bourgeois-démocratique suisse',
       'Parti évangélique suisse', 'Lega dei Ticinesi',
       'Christlich-soziale Partei Obwalden', 'sans parti',
       'Mouvement Citoyens Genevois', 'Grüne (Basels starke Alternative)',
       'Parti Suisse du travail'], dtype=object)

In [10]:
df_active_people[df_active_people['PartyAbbreviation'] == 'PLD']

,AdditionalActivity,AdditionalMandate,CantonAbbreviation,CantonName,Citizenship,CouncilAbbreviation,CouncilName,DateElection,DateJoining,DateLeaving,DateOath,DateResignation,IdPredecessor,Mandates,ParlGroupAbbreviation,ParlGroupFunctionText,ParlGroupName,PartyAbbreviation,PartyName,DateOfBirth,DateOfDeath,FirstName,GenderAsString,LastName,MaritalStatusText,MilitaryRankText,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,TitleText,PartyColor
1,Vize-Präsident der Parlamentarischen Gruppe fü...,NaN,BS,Bâle-Ville,Bâle (BS),CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,NaN,"Weiterer Bürgerrat: von 1989 bis 1991, Grosser...",RL,Membre,Groupe libéral-radical,PLD,Parti libéral démocrate,1951-01-15T00:00:00,NaN,Christoph,m,Eymann,marié(e),Capitaine,D,3,Eymann,2287.0,74,Bâle-Ville,Bâle,Dr. iur.,1


## Transform the Birthday into Age Range



In [11]:
df_active_people['DateOfBirth'] = pd.to_datetime(df_active_people['DateOfBirth'])
df_active_people['DateOfBirth'].head()

1   1951-01-15
2   1953-10-31
3   1950-12-01
4   1952-08-26
5   1942-05-07
Name: DateOfBirth, dtype: datetime64[ns]

In [12]:
now = pd.Timestamp(dt.datetime.now())
df_active_people['DateOfBirth'] = df_active_people['DateOfBirth'].where(df_active_people['DateOfBirth'] < now, df_active_people['DateOfBirth'] -  np.timedelta64(100, 'Y'))
df_active_people['age'] = (now - df_active_people['DateOfBirth']).astype('<m8[Y]')    # 3

In [13]:
df_active_people[['DateOfBirth', 'age']].head()

,DateOfBirth,age
1,1951-01-15,66.0
2,1953-10-31,63.0
3,1950-12-01,66.0
4,1952-08-26,64.0
5,1942-05-07,74.0


We define the categories of age as:
1. 20-
2. 20-30
3. 30-40
4. 40-50
5. 50-60
6. 60-70
5. 70+

In [14]:
def age_category_text(age):
    if age < 20:
        return "-20"
    elif age < 30:
        return "20-29"
    elif age < 40:
        return "30-39"
    elif age < 50:
        return "40-49"
    elif age < 60:
        return "50-59"
    elif age < 70:
        return "60-69"
    else:
        return "70+"
    
def age_category(age):
    if age < 20:
        return 0
    elif age < 30:
        return 1
    elif age < 40:
        return 2
    elif age < 50:
        return 3
    elif age < 60:
        return 4
    elif age < 70:
        return 5
    else:
        return 6

In [15]:
df_active_people['AgeCategory'] = df_active_people['age'].apply(lambda x: age_category(x))
df_active_people['AgeCategoryText'] = df_active_people['age'].apply(lambda x: age_category_text(x))

df_active_people['AgeCategory'].head()

1    5
2    5
3    5
4    5
5    6
Name: AgeCategory, dtype: int64

## Write the DF into a file

In [16]:
df_active_people.head()

,AdditionalActivity,AdditionalMandate,CantonAbbreviation,CantonName,Citizenship,CouncilAbbreviation,CouncilName,DateElection,DateJoining,DateLeaving,DateOath,DateResignation,IdPredecessor,Mandates,ParlGroupAbbreviation,ParlGroupFunctionText,ParlGroupName,PartyAbbreviation,PartyName,DateOfBirth,DateOfDeath,FirstName,GenderAsString,LastName,MaritalStatusText,MilitaryRankText,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,TitleText,PartyColor,age,AgeCategory,AgeCategoryText
1,Vize-Präsident der Parlamentarischen Gruppe fü...,NaN,BS,Bâle-Ville,Bâle (BS),CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,NaN,"Weiterer Bürgerrat: von 1989 bis 1991, Grosser...",RL,Membre,Groupe libéral-radical,PLD,Parti libéral démocrate,1951-01-15,NaN,Christoph,m,Eymann,marié(e),Capitaine,D,3,Eymann,2287.0,74,Bâle-Ville,Bâle,Dr. iur.,1,66.0,5,60-69
2,Co-Präsident der Parlamentarischen Gruppe Elek...,NaN,AG,Argovie,"Rothrist (AG),Wattwil (SG)",CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,NaN,Legislative des Kantons (Grossrat): von März 1...,V,Membre,Groupe de l'Union démocratique du Centre,UDC,Union Démocratique du Centre,1953-10-31,NaN,Ulrich,m,Giezendanner,veuf/veuve,Soldat,D,3,Giezendanner,2292.0,91,Argovie,Rothrist,NaN,2,63.0,5,60-69
3,NaN,Präsident der SVP Schweiz: vom Januar 1996 bis...,ZH,Zurich,"Hinwil (ZH),Adelboden (BE)",CF,Conseil fédéral,2008-12-10T00:00:00,2008-12-10T00:00:00,NaN,2008-12-10T00:00:00,NaN,cef9ef51-4e74-4ef8-b9da-d39c33caef23,Exekutive der Gemeinde (Gemeinderat Hinwil): v...,NaN,NaN,NaN,UDC,Union Démocratique du Centre,1950-12-01,NaN,Ueli,m,Maurer,marié(e),Major,D,6,Maurer Ueli,2310.0,146,Zurich,Wetzikon (ZH),-,2,66.0,5,60-69
4,NaN,NaN,SG,St-Gall,Appenzell (AI),CE,Conseil des Etats,2015-11-15T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,NaN,Legislative der Gemeinde St. Gallen: von 1977 ...,S,Membre,Groupe socialiste,PSS,Parti socialiste suisse,1952-08-26,NaN,Paul,m,Rechsteiner,marié(e),NaN,D,NaN,Rechsteiner Paul,2143.0,172,St-Gall,St. Gall,lic. iur.,3,64.0,5,60-69
5,NaN,Vizepräsident SVP Bezirk Laufenburg: von 1985 ...,AG,Argovie,Oberhof (AG),CN,Conseil national,2015-10-18T00:00:00,2015-11-30T00:00:00,NaN,2015-11-30T00:00:00,NaN,NaN,NaN,V,Membre,Groupe de l'Union démocratique du Centre,UDC,Union Démocratique du Centre,1942-05-07,NaN,Maximilian,m,Reimann,NaN,Capitaine,D,NaN,Reimann Maximilian,2147.0,173,Argovie,Laufenburg,Dr. iur.,2,74.0,6,70+


In [17]:
json_active_people = {'nodes': [], 'links': []}
nbr_active = len(df_active_people['AdditionalMandate'])
for i in range(len(df_active_people['AdditionalMandate'])):
    person = {}
    person['id'] = i+1
    for j in df_active_people.columns:
        if j == 'PartyColor' or j== 'age' or j == 'AgeCategory' or j == 'PersonNumber' or j == 'PersonIdCode':
            person[j] = int(df_active_people[j][i+1])
        elif j == 'DateOfBirth':
            person[j] = df_active_people[j][i+1].strftime('%d.%m.%Y')
        else:
            person[j] = df_active_people[j][i+1]          
    
    json_active_people['nodes'].append(person)
    for k in range(0):
        link = {'source': random.randint(1, nbr_active) , 'target': random.randint(1, nbr_active), 'value': random.randint(1, 5)}
        json_active_people['links'].append(link)

In [18]:
json_active_people

{'links': [],
 'nodes': [{'AdditionalActivity': 'Vize-Präsident der Parlamentarischen Gruppe für Tourismus',
   'AdditionalMandate': 'NaN',
   'AgeCategory': 5,
   'AgeCategoryText': '60-69',
   'CantonAbbreviation': 'BS',
   'CantonName': 'Bâle-Ville',
   'Citizenship': 'Bâle (BS)',
   'CouncilAbbreviation': 'CN',
   'CouncilName': 'Conseil national',
   'DateElection': '2015-10-18T00:00:00',
   'DateJoining': '2015-11-30T00:00:00',
   'DateLeaving': 'NaN',
   'DateOath': '2015-11-30T00:00:00',
   'DateOfBirth': '15.01.1951',
   'DateOfDeath': 'NaN',
   'DateResignation': 'NaN',
   'FirstName': 'Christoph',
   'GenderAsString': 'm',
   'IdPredecessor': 'NaN',
   'LastName': 'Eymann',
   'Mandates': 'Weiterer Bürgerrat: von 1989 bis 1991, Grosser Rat des Kantons Basel-Stadt: von 1984 bis 1995',
   'MaritalStatusText': 'marié(e)',
   'MilitaryRankText': 'Capitaine',
   'NativeLanguage': 'D',
   'NumberOfChildren': 3.0,
   'OfficialName': 'Eymann',
   'ParlGroupAbbreviation': 'RL',
   'P

In [19]:
with open('../viz/data/active.json', 'w') as ff:
    json.dump(json_active_people, ff, indent=2)

In [20]:
df_active_people.drop(['AdditionalActivity', 'AdditionalMandate', 'Mandates'], axis=1).to_csv('data/Active_People.csv', encoding='utf-8', index=False)
       

## Get the pictures =)

In [21]:
url_portrait = "https://www.parlament.ch/sitecollectionimages/profil/portrait-260/"

In [22]:
for i in df_active_people.index:
    person_id_code = int(df_active_people['PersonIdCode'][i])
    f = open('../viz/data/portraits/' + str(person_id_code) + '.jpg','wb')
    f.write(requests.get(url_portrait + str(person_id_code) + '.jpg').content)
    f.close()
    